# Классы, наследование, агрегация, композиция и примеси 

## Класс 

Класс в Ruby - это объект специального типа. Для его создания достаточно простой конструкции

In [3]:
class ExampleClass
end

In [7]:
ExampleClass.class # Тип объекта.

Class

Создание нового экземпляра - через метод `:new`

In [11]:
example_class = ExampleClass.new

#<ExampleClass:0x00007f8d758a2988>

## Атрибуты класса

Можно объявлять переменные уровня класса (статические) и уровня объекта. Их нужно объявлять в конструкторе. **Конструктор называется `initialize`**.

In [23]:
class ExampleClass
  def initialize
    @@static = 'static'
    @not_static = 'not_static'
  end
  
  def update_static(v)
    @@static = v
  end
  
  def print_static
    puts @@static
  end
  
  def update_not_static(v)
    @not_static = v
  end
  
  def print_not_static
    puts @not_static
  end
end

:print_not_static

Так делать при этом нельзя

In [24]:
ExampleClass.static

NoMethodError: undefined method `static' for ExampleClass:Class

In [25]:
a = ExampleClass.new
b = ExampleClass.new

#<ExampleClass:0x00007f8d79895f90 @not_static="not_static">

In [27]:
a.update_not_static(10)
a.print_not_static
b.print_not_static

10
"not_static"


"not_static"

In [29]:
b.update_static(20)
b.print_static
a.print_static

20
20


20

"Статические" переменные разделяют свое значение между всеми экземплярами класса, а каждый экземпляр может это общее значение менять. Такая организация кода приводит к большому количеству ошибок при его усложнении, поэтому **пользоваться `@@` крайне не рекомендую!**

Для того, чтобы переменные уровня объекта были доступны извне, нужно описать соответствующие методы. По соглашению методы называются так же, как и соответствующие им атрибуты (только без `@`):

In [32]:
class ExplicitAttrs
  def initialize
    @a = 10
  end
  
  # Геттер.
  def a
    @a
  end
  
  # Сеттер.
  def a=(v)
    @a = v
  end
end

ea = ExplicitAttrs.new

puts ea.a
ea.a = 100
puts ea.a

10
100


Если мы хотим, чтобы атрибут был доступен только для чтения или только для записи, мы должны определить один соответствующий метод - или геттер, или сеттер.

Чтобы не мучиться с большим количеством однотипного кода и не загромождать класс, можно пользовать встроенными макросами:

* `attr_reader` создаст только геттер;
* `attr_writer` создаст только сеттер;
* `attr_accessor` можно использовать, если нужен и геттер, и сеттер (то же самое, что написать сначала `attr_reader`, затем `attr_writer`);

In [35]:
class ImplicitAttrs
  attr_reader :a
  attr_writer :b
  attr_accessor :c
  
  # Этот метод вызовется, если распечатать объект через p.
  def inspect
    "a: #{@a}, b: #{@b}, c: #{@c}"
  end
end

:inspect

При этом **необязательно объявлять атрибуты в конструкторе (методе `initialize`)**. Начальное значение будет `nil`, а потом 

In [36]:
ia = ImplicitAttrs.new

a: , b: , c: 

In [38]:
ia.a.nil?

true

In [43]:
ia.a = 1

NoMethodError: undefined method `a=' for a: , b: 10, c: 100:ImplicitAttrs
Did you mean?  a

In [39]:
ia.b

NoMethodError: undefined method `b' for a: , b: , c: :ImplicitAttrs
Did you mean?  b=

In [40]:
ia.b = 10
ia

a: , b: 10, c: 

In [41]:
ia.c = 100
ia.c

100

In [42]:
ia

a: , b: 10, c: 100

Обычно **нигде, кроме конструктора, не пользуются `@`, а создают `attr_*` в зависимости от того, что нужно**!

## Инкапсуляция

Для инкапсуляции предусмотрены специальные ключевые слова (читай - методы):

In [51]:
class EncapsulationExample
  def public_method
    puts 'public'
  end
  
  protected
  
  def protected1
    puts 'protected1'
  end
  
  def protected2
    puts 'protected2'
  end
  
  private
  
  def private1
    puts 'private1'
  end
  
  public
  
  def also_public
    puts 'also public'
  end
end

:also_public

In [46]:
ee = EncapsulationExample.new

#<EncapsulationExample:0x00007f8d7808d0b0>

In [47]:
ee.public_method

public


In [48]:
ee.also_public

also public


In [49]:
ee.protected1

NoMethodError: protected method `protected1' called for #<EncapsulationExample:0x00007f8d7808d0b0>
Did you mean?  protected2

In [50]:
ee.private1

NoMethodError: private method `private1' called for #<EncapsulationExample:0x00007f8d7808d0b0>

Тем не менее, ограничение достаточно легко обходится с помощью встроенного механизма обмена сообщениями (вообще говоря, вызов метода в Ruby - это и есть отправка сообщения).

In [52]:
ee.send(:private1)

private1


## Наследование 

In [8]:
ExampleClass.superclass # Родительский класс.

Object

In [9]:
ExampleClass.ancestors # Цепочка наследования

[ExampleClass, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

In [1]:
class Figure
  attr_accessor :width, :length
  
  def initialize(width, length)
    @width = width
    @length = length
  end
  
  def square
    raise NotImplementedError, 'Method square is not implemented'
  end
  
  def volume
    raise NotImplementedError, 'Method volume is not implemented'
  end
end

class Rectangle < Figure
  def square
    width * length
  end
end

class Square < Rectangle
  def initialize(s)
    super(s, s)
  end
end

:initialize

In [2]:
r = Rectangle.new(2, 7)
s = Square.new(10)

#<Square:0x00007fbd7195c1e0 @width=10, @length=10>

In [3]:
r.square

14

In [4]:
r.volume

NotImplementedError: Method volume is not implemented

In [5]:
s.square

100

В Ruby **применяется одиночное наследование**.

Наследование как концепт хорошо в том случае, когда наследник - это **более специфичная версия родителя**, иногда оно **не нужно**.

Допустим, мы хотим добавить в иерархию фигуры с объемом. Начнем с куба. Это можно сделать так:

In [10]:
class Cube < Square
  attr_accessor :height
  
  def initialize(s)
    super
    @height = s
  end
  
  def square
    6 * super
  end
  
  def volume
    width**3
  end
end

:volume

In [11]:
c = Cube.new(5)

#<Cube:0x00007fbd702f5fc8 @width=5, @length=5, @height=5>

In [12]:
c.square

150

In [13]:
c.volume

125

Теперь попробуем добавить прямоугольный параллелепипед по аналогии:

In [2]:
class RectangularParallelepiped < Rectangle
  attr_accessor :height
  
  def initialize(width, length, height)
    super(width, length)
    @height = height
  end
  
  def square
    2 * (width * height + width * length + length * height)
  end
  
  def volume
    width * length * height
  end
end

:volume

In [4]:
rp = RectangularParallelepiped.new(1, 2, 3)

#<RectangularParallelepiped:0x00007fc88168c8a0 @width=1, @length=2, @height=3>

In [5]:
rp.square

22

In [6]:
rp.volume

6

Налицо дублирование кода. Попробуем переделать иерархию

In [7]:
class FigureV2
  def square
    raise NotImplementedError, 'Method square is not implemented'
  end
  
  def volume
    raise NotImplementedError, 'Method volume is not implemented'
  end
end

class FlatFigure < FigureV2
  attr_accessor :width, :length
  
  def initialize(width, length)
    @width = width
    @length = length
  end
end

class VolumeFigure < FlatFigure
  attr_accessor :height
  
  def initialize(width, length, height)
    super(width, length)
    @height = height
  end
end

:initialize

Концептуально так более правильно, попробуем реализовать прямоугольник и квадрат

In [8]:
class RectangleV2 < FlatFigure
  def square
    width * length
  end
end

class SquareV2 < Rectangle
  def initialize(s)
    super(s, s)
  end
end

:initialize

In [9]:
r2 = RectangleV2.new(4, 7)
s2 = SquareV2.new(6)

#<SquareV2:0x00007fc8838b3a00 @width=6, @length=6>

In [10]:
r2.square

28

In [11]:
s2.square

36

Пока все идет неплохо, но вот что делать, если мы хотим реализовать куб и параллелепипед? С одной стороны, нужно наследоваться от `VolumeFigure`, чтобы два раза не определять атрибут `height`, с другой - от `Rectangle`, чтобы не определять два раза вычисление площади, с третьей - вообще от `Square`, чтобы сохранить удобную сигнатуру конструктора.

Выход есть - **примеси**!

In [12]:
module Squareable
  def square
    width * length
  end
end

module Volumeable
  def volume
    width * length * height 
  end
end

:volume

In [22]:
class CubeV2 < VolumeFigure
  include Squareable, Volumeable
  
  def initialize(s)
    super(s, s, s)
  end
  
  def square
    6 * super
  end
end

class RectangularParallelepipedV2 < VolumeFigure
  include Volumeable
  
  def square
    2 * (width * height + width * length + length * height)
  end
end

:square

In [23]:
c2 = CubeV2.new(5)
rp2 = RectangularParallelepipedV2.new(1, 2, 3)

#<RectangularParallelepipedV2:0x00007fc8813e72d0 @width=1, @length=2, @height=3>

In [24]:
c2.square

150

In [19]:
c2.volume

125

In [20]:
rp2.square

22

In [21]:
rp2.volume

6

Примеси - это способ **горизонтально распространять код** и их нужно предпочитать наследованию когда это возможно.

Чем еще можно заменить наследование?

## Агрегация и композиция 

И агрегация, и композиция - это конструирование объекта из нескольких. Иными словами, вы не строите иерархию с кучей методов и ничего не примешиваете, а просто делаете специальный объект для какой-то специальной цели и используете его внутри других объектов.

Классический пример - логирование. Вряд ли имеет хоть какой-то смысл наследоваться от логгера или шарить код логирования через примесь. Помимо концептуальной некорректности абстракции это еще и сложно тестировать и поддерживать.

Посмотрим, как можно реализовать агрегацию и композицию на примере логера.

In [38]:
class MyLogger
  def log(message)
    puts "[#{Time.now.strftime("%Y-%m-%d %H:%M:%S")}] #{message}"
  end
end

:log

In [39]:
MyLogger.new.log('Привет')

[2020-10-28 10:54:13] Привет


Посмотрим, что можно сделать с использованием композиции:

In [42]:
class UserServiceComposition
  def initialize
    @logger = MyLogger.new
  end
  
  def login
    logger.log('User login')
  end
  
  def logout
    logger.log('User logout')
  end
  
  private
  
  attr_reader :logger
  
  public
  
  class << self
    def login
      new.login
    end
    
    def logout
      new.logout
    end
  end
end

:logout

In [43]:
UserServiceComposition.login

[2020-10-28 10:58:04] User login


In [44]:
UserServiceComposition.logout

[2020-10-28 10:58:12] User logout


Класс **сам управляет своими зависимостями**.

То же самое с использованием агрегации:

In [64]:
class UserServiceAggregation
  def initialize(logger)
    @logger = logger
  end
  
  def login
    logger.log('User login')
  end
  
  def logout
    logger.log('User logout')
  end
  
  private
  
  attr_reader :logger
end

class DependencyInjector
  @instance = new

  private_class_method :new

  def initialize
    @dependecies = Hash.new
    @dependecies[MyLogger] = MyLogger.new
    @dependecies[UserServiceAggregation] = UserServiceAggregation.new(@dependecies[MyLogger])
  end
  
  attr_reader :dependecies
  
  public
  
  class << self    
    def get(class_name)
      instance.dependecies[class_name]
    end
  end
end

:get

In [65]:
service = DependencyInjector.get(UserServiceAggregation)

#<UserServiceAggregation:0x00007fc8828b8e20 @logger=#<MyLogger:0x00007fc8828b8e48>>

In [66]:
service.login

[2020-10-28 11:09:19] User login


Зависимостями **управляет кто-то извне**.

В конечном счете агрегация удобнее, потому что позволяет легко подменять реальные классы (это ведь может быть не только логер, но и сервис для работы с БД, например) на какие-то другие.

Если вы соблюдаете интерфейс, вы можете:

* перейти с одного источника данных на другой;
* сделать "мок" - более простую реализацию, например, для окружения разработчика или пока ждете реализации сервисов со стороны коллег;
* легко тестировать.

## Enumerable, Enumerator 

`Enumerable` - это примесь, в которой реализована куча полезных методов по работе с коллекциями. Единственное, что нужно примеси - это метод для итерированию по коллекции, по умолчанию - `each`. Самое смешное, что объект, к которому мы примешиваем `Enumerable`, строго говоря не должен быть именно коллекцией (массивом, например).

In [69]:
class MyCollection
  include Enumerable
  
  attr_reader :name, :surname
  
  def initialize
    @name = 'Иван'
    @surname = 'Иванов'
  end
  
  def each
    yield name
    yield surname
  end
end

:each

In [71]:
MyCollection.new.map { |x| x }

["Иван", "Иванов"]

In [72]:
MyCollection.new.take(1)

["Иван"]

`Enumerator` - это то же самое, что и `Enumerable`, только его можно создавать как объект (потому что это и есть объект).

In [75]:
e = Enumerator.new do |yielder|
  yielder.yield 'Иван'
  yielder.yield 'Иванов'
end

e.take(1)

["Иван"]

Пропробуем решить задачу

Найти первый член последовательности `y = n / (n**2 + 2)`, для которого y < ξ. Определить, как изменяется число итераций при изменении
точности.

In [91]:
eps = 0.001

enum = Enumerator.new do |y|
  n = 1
  
  loop do
    y << n / (n**2 + 2).to_f
    n += 1
  end
end

#<Enumerator: #<Enumerator::Generator:0x00007fc8817ad130>:each>

In [94]:
enum.find { |y| y < eps }

0.000999998000004

## Ленивые вычисления 

Классический пример.

Просто 5 чисел:

In [95]:
(1..Float::INFINITY).take(5)

[1, 2, 3, 4, 5]

Пять четных чисел:

In [98]:
(1..Float::INFINITY).filter(&:even?).take(5)

Interrupt: 

Пять ленивых четных чисел

In [100]:
(1..Float::INFINITY).lazy.filter(&:even?).take(5).to_a

[2, 4, 6, 8, 10]

Как это работает

In [104]:
(1..10).map do |x|
  puts "Map 1 #{x}"
  x + 1
end.map do |x|
  puts "Map 2 #{x}"
  x * 2
end

Map 1 1
Map 1 2
Map 1 3
Map 1 4
Map 1 5
Map 1 6
Map 1 7
Map 1 8
Map 1 9
Map 1 10
Map 2 2
Map 2 3
Map 2 4
Map 2 5
Map 2 6
Map 2 7
Map 2 8
Map 2 9
Map 2 10
Map 2 11


[4, 6, 8, 10, 12, 14, 16, 18, 20, 22]

In [106]:
(1..10).lazy.map do |x|
  puts "Map 1 #{x}"
  x + 1
end.map do |x|
  puts "Map 2 #{x}"
  x * 2
end.force

Map 1 1
Map 2 2
Map 1 2
Map 2 3
Map 1 3
Map 2 4
Map 1 4
Map 2 5
Map 1 5
Map 2 6
Map 1 6
Map 2 7
Map 1 7
Map 2 8
Map 1 8
Map 2 9
Map 1 9
Map 2 10
Map 1 10
Map 2 11


[4, 6, 8, 10, 12, 14, 16, 18, 20, 22]

Иными словами, `lazy` прокидывает каждое значение сразу через все блоки, а обычный энумератор - "накапливает" у начала каждого блока.

## Proc, блок и lambda

В третьей часть шестой лабораторной работы вам необходимо сделать так, чтобы некий кусок кода (в зависимости от задания) мог быть передан и как блок, и как Proc/lambda.

Попробуем решить такую задачу

Составить метод scale отыскания масштаба графического (выведенного в текстовом режиме) изображения функции f(x) на экране размером B единиц растра по формуле `M = B / max(f(x))`. В основной программе использовать метод для отыскания масштаба функций `x · sin(x)` и `tg(x)`, при `−2 < x < 2`. Реализовать вызов метода двумя способами: в виде передаваемого lambda-выражения и в виде блока.

Теперь эталонное решение будет таким

In [107]:
def find_scale(left, right, b)
  max = (left..right).step(0.01).map { |x| yield x }.max
  b / max if max != 0 || nil
end

:find_scale

Вызов с использованием блока

In [108]:
find_scale(-2, 2, 10) { |x| Math.sin(x) * x }

5.4987508514730825

Вызов с использованием `proc`

In [109]:
find_scale(-2, 2, 10, &proc { |x| Math.sin(x) * x })

5.4987508514730825

Все это работает через один и тот же `yield`, настолько быстро, насколько это возможно, и максимально идиоматично.